# **Tuto : Construire un Chatbot Médical Basé sur  TF-IDF et Similarité Cosinus**
**Enseignante:** Asmaa BENGUEDDACH

**Niveau:** Master

**Durée:** (3 heures)

---

**Objectif**
Créer un chatbot médical fonctionnel qui utilise une base de données de questions/réponses nettoyées pour les calculs (TF-IDF et similarité cosinus), tout en affichant les questions et réponses originales pour l'utilisateur.

---

## **1. Modélisation avec TF-IDF**

#### **1.1 Comprendre TF-IDF (Term Frequency-Inverse Document Frequency)**

**Qu'est-ce que TF-IDF ?**
- TF-IDF est une méthode utilisée en traitement automatique du langage naturel (NLP) pour évaluer l'importance d'un mot dans un document, relativement à un corpus de documents.
- Elle transforme les textes (les questions nettoyées) en vecteurs numériques pour pouvoir les comparer mathématiquement.

### **TF (Term Frequency) :**

C'est la fréquence d'apparition d'un mot dans un document, calculée par la formule suivante :

Par exemple, si le mot "diabète" apparaît 5 fois dans un document de 100 mots, le TF est :

> **`TF(t) = Nombre de fois que le mot t apparaît dans un documnent / Nombre total de mots dans le document `**

- Par exemple, si le mot **"diabète"** apparaît 5 fois dans un document de 100 mots, le TF est :

TF(diabetes) = 5/100 = 0.05

- **IDF (Inverse Document Frequency) :**

C'est une mesure de la rareté d'un mot dans l'ensemble des documents. Si un mot est très fréquent dans plusieurs documents, son importance diminue.

> **`IDF(t) = log(Nombre total de documents / Le nombre de document contenant le document t`**)

-  Par exemple, si le mot "**diabète**" est présent dans 3 documents sur 100, alors :

IDF(diabetes) = log (100/3) = 1.52.

- **TF-IDF :**

On combine TF et IDF pour obtenir une valeur pondérée :

> **`TF-IDF(t) = TF(t) x IDF(t)`**





- Les mots fréquents dans un document mais rares dans le corpus ont une valeur TF-IDF élevée.

- **Pourquoi utiliser TF-IDF ?**

  - Permet d'éliminer l'impact des mots courants (ex. : "le", "et", "de") qui apparaissent partout.

  - De mettre en valeur les mots significatifs pour un document spécifique.

En pratique, TF-IDF est utilisé pour transformer les questions (ou documents) en vecteurs numériques.

## **Exemple complet : TF-IDF**

### Étape 1 : Importations nécessaires
Assurez-vous d’avoir les bibliothèques nécessaires installées et prêtes à l’emploi.

In [ ]:
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

- `spacy` : Pour effectuer la lemmatisation et le prétraitement.
- `sklearn.feature_extraction.text` : Pour calculer la matrice TF-IDF.
- `sklearn.metrics.pairwise` : Pour calculer la similarité cosinus.
- `pandas` : Pour manipuler les données dans un DataFrame.

### Étape 2 : Prétraitement des données

a. Charger le modèle SpaCy

Pour la lemmatisation en français, utilisez le modèle fr_core_news_sm de SpaCy.

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 82.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load("fr_core_news_sm")

b. Fonctions de nettoyage et de lemmatisation

Créez deux fonctions :

1. **Nettoyage basique** : Supprimer les caractères spéciaux, convertir en minuscules, etc.
2. **Lemmatisation** : Convertir les mots en leur forme canonique.

In [ ]:
# Nettoyage de base : suppression des caractères inutiles
def nettoyer_texte(texte):
    texte = re.sub(r"[^a-zA-Z0-9éèêçàùâû\s]", "", texte.lower())  # Minuscules et suppression des caractères spéciaux
    return texte

# Lemmatisation et suppression des stopwords
def lemmatize_texte(texte):
    doc = nlp(texte)
    lemmes = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct  # Supprime les stopwords et la ponctuation
    ]
    return " ".join(lemmes)

### Étape 3 : Créer le DataFrame
Préparez un dataset d'exemple contenant des questions et réponses :

In [ ]:
data = {
    "question": [
        "Quelles sont les causes du diabète ?",
        "Quels sont les symptômes du diabète ?",
        "Comment traiter l'hypertension artérielle ?"
    ],
    "answer": [
        "Les causes incluent une mauvaise alimentation et des prédispositions génétiques.",
        "Les symptômes incluent une soif excessive et une fatigue inhabituelle.",
        "Le traitement inclut des médicaments et une modification du mode de vie."
    ]
}

df = pd.DataFrame(data)

# Appliquer le nettoyage et la lemmatisation
df["Cleaned_Question"] = df["question"].apply(nettoyer_texte)
df["Lemmatized_Question"] = df["Cleaned_Question"].apply(lemmatize_texte)

In [ ]:
df

,question,answer,Cleaned_Question,Lemmatized_Question
0,Quelles sont les causes du diabète ?,Les causes incluent une mauvaise alimentation ...,quelles sont les causes du diabète,cause diabète
1,Quels sont les symptômes du diabète ?,Les symptômes incluent une soif excessive et u...,quels sont les symptmes du diabète,symptme diabète
2,Comment traiter l'hypertension artérielle ?,Le traitement inclut des médicaments et une mo...,comment traiter lhypertension artérielle,traiter lhypertension artériel


### Étape 4 : Calculer la matrice TF-IDF

Initialisez le **TfidfVectorizer** et calculez la matrice TF-IDF en utilisant les questions lemmatisées.

In [ ]:
# Initialisation du TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["Lemmatized_Question"])

In [ ]:
print(tfidf_matrix)

  (0, 1)	0.7959605415681652
  (0, 2)	0.6053485081062916
  (1, 2)	0.6053485081062916
  (1, 4)	0.7959605415681652
  (2, 5)	0.5773502691896257
  (2, 3)	0.5773502691896257
  (2, 0)	0.5773502691896257



- **indice_ligne** : Le numéro de ligne où se trouve l'élément non nul.
- **indice_colonne** : Le numéro de colonne où se trouve l'élément non nul.
- **valeur** : La valeur réelle de l'élément non nul.

### Exemple

Prenons la première ligne comme exemple :

(0, 1) 0.7959605415681652


Cela signifie que l'élément situé à la ligne 0 et à la colonne 1 de la matrice a une valeur de `0.7959605415681652`. Tous les autres éléments de la matrice qui ne sont pas explicitement listés sont considérés comme étant nuls.

- `tfidf_matrix` : Une matrice sparse où chaque ligne représente une question et chaque colonne représente un mot du vocabulaire.

In [ ]:
# Extraction des noms des colonnes (mots-clés) et des lignes (documents)
mots_cles = vectorizer.get_feature_names_out()
documents = [f"Question {i+1}" for i in range(tfidf_matrix.shape[0])]

# Conversion en DataFrame Pandas
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), index=documents, columns=mots_cles)

# Affichage
print("Matrice TF-IDF :")
print(df_tfidf)

Matrice TF-IDF :
            artériel     cause   diabète  lhypertension   symptme  traiter
Question 1   0.00000  0.795961  0.605349        0.00000  0.000000  0.00000
Question 2   0.00000  0.000000  0.605349        0.00000  0.795961  0.00000
Question 3   0.57735  0.000000  0.000000        0.57735  0.000000  0.57735


Ces résultats représentent une matrice TF-IDF, où :

- **Les colonnes** (`artériel, cause, diabète, lhypertension, symptme, traiter`) sont les mots-clés (caractéristiques) extraits des documents.
- **Les lignes** (`Document 1, Document 2, Document 3`) représentent les documents (ou questions dans le contexte d'un chatbot).
- **Les valeurs** dans la matrice indiquent l'importance relative (pondérée par `TF-IDF`) de chaque mot-clé dans un document particulier.

#### **Analyse des résultats :**

1. Analyse des scores TF-IDF
- Document 1 (**Question 1**)

Les mots les plus importants sont cause (**0.795961**) et diabète (**0.605349**).

> Cela suggère que ce document parle principalement des causes du diabète.
Les autres mots-clés comme artériel, lhypertension, et traiter n'apparaissent pas dans ce document (**score 0**).

- Document 2 (**Question 2**)

> Les mots les plus importants sont symptme (**0.795961**) et diabète (**0.605349**).
Cela indique que ce document se concentre sur les symptômes du diabète.
Les mots artériel, lhypertension, et traiter ne sont pas présents.

- Document 3 (**Question 3**)

> Les mots les plus importants sont artériel (**0.57735**), lhypertension (**0.57735**), et traiter (**0.57735**).
Cela suggère que ce document traite principalement de l'hypertension artérielle et des traitements associés.
Les mots cause, diabète, et symptme ne figurent pas dans ce document.

2. Interprétations générales

**Importance des mots dans chaque document**
 - Le score élevé d'un mot dans un document indique que :
  - Ce mot est fréquent dans le document (forte TF).
  - Ce mot est relativement rare dans les autres documents (forte IDF)

**Spécificité des documents**
 - Les scores TF-IDF montrent que les documents sont bien différenciés :
  - Document 1 parle des causes du diabète.
  - Document 2 se concentre sur les symptômes du diabète.
  - Document 3 traite de l'hypertension artérielle et des traitements.

**Absence de certains mots**
- Si un mot a un score 0, cela signifie qu'il n'apparaît pas dans ce document.
- Par exemple, artériel et traiter sont absents des documents 1 et 2.

## **2. Comprendre la Similarité Cosinus**

La similarité cosinus permet de trouver la question existante la plus proche de la requête utilisateur

**Comment fonctionne t il ?**

La similarité cosinus mesure la similarité entre deux vecteurs numériques, en calculant le cosinus de l'angle entre eux. La valeur varie entre 0 (pas de similarité) et 1 (vecteurs identiques).

La formule est donnée par :


> **Similarité cosinus (A,B) = Produit_scalaire(A,B)/||A|| x ||B||**

- Explication Géométrique:

  - Si deux vecteurs (représentant des textes) pointent dans la même direction, leur similarité cosinus est proche de 1.
  - Si les vecteurs sont perpendiculaires (aucune similitude), la valeur est 0.

**Pourquoi utiliser la similarité cosinus ?**

- Elle permet de comparer efficacement les documents ou questions transformés en vecteurs TF-IDF :

- Trouver la question la plus proche de celle posée par l'utilisateur.
Calculer un score de similarité pour déterminer si deux textes se ressemblent.


## **Exemple complet : La similarité cosinus**

Créez une fonction pour :

1. Prendre une question utilisateur.
2. La nettoyer et la lemmatiser pour la transformer en un vecteur TF-IDF.
3. Calculer la similarité cosinus entre cette question et toutes les questions du dataset.
4. Retourner la question la plus similaire et la réponse correspondante.

In [ ]:
def trouver_question_similaire(user_question):
    # Nettoyer et lemmatiser la question utilisateur
    user_question_cleaned = nettoyer_texte(user_question)
    user_question_lemmatized = lemmatize_texte(user_question_cleaned)

    # Transformer la question utilisateur en vecteur TF-IDF
    user_vec = vectorizer.transform([user_question_lemmatized])

    # Calculer la similarité cosinus entre la question utilisateur et les questions du dataset
    similarities = cosine_similarity(user_vec, tfidf_matrix)
    print("Similarités :", similarities)

    # Identifier l'index de la question la plus similaire
    best_match_idx = similarities.argmax()
    best_match_score = similarities[0, best_match_idx]

    # Retourner la question et la réponse correspondantes
    return df["question"].iloc[best_match_idx], df["answer"].iloc[best_match_idx], best_match_score

Tester avec une Question Utilisateur

- Ajoutez un test pour poser une question et afficher la question similaire ainsi que la réponse.

In [ ]:
# Exemple de question utilisateur
user_question = "Quels sont les signes du diabète ?"

# Trouver la question similaire et sa réponse
question_similaire, reponse_similaire, score = trouver_question_similaire(user_question)

# Afficher les résultats
print("Question utilisateur :", user_question)
print("Question similaire trouvée :", question_similaire)
print("Réponse correspondante :", reponse_similaire)
print("Score de similarité :", score)

Similarités : [[0.60534851 0.60534851 0.        ]]
Question utilisateur : Quels sont les signes du diabète ?
Question similaire trouvée : Quelles sont les causes du diabète ?
Réponse correspondante : Les causes incluent une mauvaise alimentation et des prédispositions génétiques.
Score de similarité : 0.6053485081062917


il y a deux questions avec le meme score ! alors qu'il a choisi la premiere

In [ ]:
def trouver_questions_similaires(user_question):
    # Nettoyer et lemmatiser la question utilisateur
    user_question_cleaned = nettoyer_texte(user_question)
    user_question_lemmatized = lemmatize_texte(user_question_cleaned)

    # Transformer la question utilisateur en vecteur TF-IDF
    user_vec = vectorizer.transform([user_question_lemmatized])

    # Calculer la similarité cosinus entre la question utilisateur et les questions du dataset
    similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()
    print("Similarités :", similarities)

    # Trouver le score maximal
    max_score = similarities.max()

    # Identifier tous les index des questions ayant le score maximal
    best_match_indices = [i for i, score in enumerate(similarities) if score == max_score]

    # Rassembler toutes les questions et réponses correspondantes
    results = []
    for idx in best_match_indices:
        question = df["question"].iloc[idx]
        answer = df["answer"].iloc[idx]
        results.append((question, answer, max_score))

    return results

In [ ]:
# Exemple de question utilisateur
user_question = "Quels sont les signes du diabète ?"

# Trouver les questions similaires et leurs réponses
resultats_similaires = trouver_questions_similaires(user_question)

# Afficher les résultats
print("Question utilisateur :", user_question)
if resultats_similaires:
    for i, (question_similaire, reponse_similaire, score) in enumerate(resultats_similaires, start=1):
        print(f"\nRésultat {i} :")
        print(f"  Question similaire trouvée : {question_similaire}")
        print(f"  Réponse correspondante : {reponse_similaire}")
        print(f"  Score de similarité : {score}")
else:
    print("Aucune question similaire trouvée.")

Similarités : [0.60534851 0.60534851 0.        ]
Question utilisateur : Quels sont les signes du diabète ?

Résultat 1 :
  Question similaire trouvée : Quelles sont les causes du diabète ?
  Réponse correspondante : Les causes incluent une mauvaise alimentation et des prédispositions génétiques.
  Score de similarité : 0.6053485081062917

Résultat 2 :
  Question similaire trouvée : Quels sont les symptômes du diabète ?
  Réponse correspondante : Les symptômes incluent une soif excessive et une fatigue inhabituelle.
  Score de similarité : 0.6053485081062917


## **Partie 2 : Projet Pratique**

### **2.1. Structure des Données**

Assurez-vous que votre fichier CSV contient :
- Question_Originale : Les questions originales.
- Réponse : Les réponses associées.
- Question_Nettoyée : Les questions nettoyées pour le traitement.


### **2.2. Charger les Données avec pandas**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Utilisez pandas pour lire le fichier CSV et extraire les colonnes nécessaires :

In [ ]:
# Test

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import spacy
import re

# Charger SpaCy
nlp = spacy.load("en_core_web_sm")

# Nettoyage des textes
#def nettoyer_texte(texte):
#    texte = re.sub(r"[^a-zA-Z0-9\s]", "", texte.lower())
#    return texte.strip()

# Lemmatisation
#def lemmatize_texte(texte):
#    doc = nlp(texte)
#    lemmes = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
#    return " ".join(lemmes)

# dataset
df = pd.read_csv('/content/drive/My Drive/SCSanté/05/medquad.csv')

# Nettoyage et lemmatisation
#df["Cleaned_Question"] = df["question"].apply(nettoyer_texte)
#df["Lemmatized_Question"] = df["Cleaned_Question"].apply(lemmatize_texte)

# TF-IDF avec des ajustements
vectorizer = TfidfVectorizer(
    stop_words="english",        # Supprime stopwords anglais
    min_df=1,                    # Inclut mots présents dans au moins 1 document
    max_df=0.8,                  # Exclut mots présents dans plus de 80% des documents
    max_features=50             # Limite à 50 mots-clés
)
tfidf_matrix = vectorizer.fit_transform(df["question"])

# Créer un DataFrame lisible
mots_cles = vectorizer.get_feature_names_out()
documents = [f"Question {i+1}" for i in range(tfidf_matrix.shape[0])]
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), index=documents, columns=mots_cles)

In [ ]:
# Affichage des mots-clés et scores
print("Matrice TF-IDF :")
df_tfidf.head()

Matrice TF-IDF :


,affected,anemia,ataxia,autosomal,cancer,causes,cell,changes,childhood,chronic,...,primary,related,research,risk,symptoms,syndrome,treatments,trials,tumors,type
Question 1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Question 2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Question 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
Question 4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Question 5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_tfidf.tail()

,affected,anemia,ataxia,autosomal,cancer,causes,cell,changes,childhood,chronic,...,primary,related,research,risk,symptoms,syndrome,treatments,trials,tumors,type
Question 16408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Question 16409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Question 16410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Question 16411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.482226,0.0,0.0,0.0
Question 16412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


#### **2.3 Calculer la Similarité Cosinus**

In [ ]:
def trouver_questions_similaires(user_question):
    # Nettoyer et lemmatiser la question utilisateur
    #user_question_cleaned = nettoyer_texte(user_question)
    #user_question_lemmatized = lemmatize_texte(user_question_cleaned)

    # Transformer la question utilisateur en vecteur TF-IDF
    user_vec = vectorizer.transform([user_question])

    # Calculer la similarité cosinus entre la question utilisateur et les questions du dataset
    similarities = cosine_similarity(user_vec, tfidf_matrix)[0]  # Flatten the array
    print("Similarités :", similarities)

    # Trouver le score maximal
    max_score = similarities.max()

    # Identifier tous les index des questions ayant le score maximal
    best_match_indices = [i for i, score in enumerate(similarities) if score == max_score]

    # Rassembler toutes les questions et réponses correspondantes
    results = []
    for idx in best_match_indices:
        question = df["question"].iloc[idx]
        answer = df["answer"].iloc[idx]
        results.append((question, answer, max_score))

    return results

In [ ]:
# Exemple de question utilisateur
user_question = "What are the causes of diabetes?"

# Trouver les questions similaires et leurs réponses
resultats_similaires = trouver_questions_similaires(user_question)

# Afficher les résultats
print("Question utilisateur :", user_question)
if resultats_similaires:
    for i, (question_similaire, reponse_similaire, score) in enumerate(resultats_similaires, start=1):
        print(f"\nRésultat {i} :")
        print(f"  Question similaire trouvée : {question_similaire}")
        print(f"  Réponse correspondante : {reponse_similaire}")
        print(f"  Score de similarité : {score}")
else:
    print("Aucune question similaire trouvée.")

Similarités : [0.         0.61368426 0.         ... 0.61629964 0.69168416 0.78955153]
Question utilisateur : What are the causes of diabetes?

Résultat 1 :
  Question similaire trouvée : What causes Diabetes ?
  Réponse correspondante : Type 1 diabetes is an autoimmune disease. In an autoimmune reaction, antibodies, or immune cells, attach to the body's own healthy tissues by mistake, signaling the body to attack them. At present, scientists do not know exactly what causes the body's immune system to attack the insulin-producing cells in the pancreas in people with type 1 diabetes. However, many believe that both genetic factors and environmental factors are involved. Studies now are underway to identify these factors and prevent type 1 diabetes in people at risk. Type 2 diabetesthe most common form of diabetesis caused by a combination of factors, including insulin resistance, a condition in which the bodys muscle, fat, and liver cells do not use insulin effectively. Type 2 diabetes d

#### 2.4. Ajouter un Seuil pour les Questions Hors Dataset

Pour gérer les cas où la question utilisateur est trop différente des questions du dataset, ajoutez un seuil de similarité.

In [ ]:
def trouver_questions_similaires(user_question, seuil_similarite):
    # Nettoyer et lemmatiser la question utilisateur
    #user_question_cleaned = nettoyer_texte(user_question)
    #user_question_lemmatized = lemmatize_texte(user_question_cleaned)

    # Transformer la question utilisateur en vecteur TF-IDF
    user_vec = vectorizer.transform([user_question])

    # Calculer la similarité cosinus entre la question utilisateur et les questions du dataset
    similarities = cosine_similarity(user_vec, tfidf_matrix)[0]  # Flatten the array

    # Trouver les index des questions avec une similarité au-dessus du seuil
    indices_valables = [i for i, score in enumerate(similarities) if score >= seuil_similarite]

    # Rassembler toutes les questions et réponses correspondantes au seuil
    results = [
        (df["question"].iloc[i], df["answer"].iloc[i], similarities[i])
        for i in indices_valables
    ]

    return results

In [ ]:
# Exemple de question utilisateur
user_question = "What are the possible long-term health complications caused by the coronavirus, including its effects on the respiratory system, cardiovascular health, and neurological functions, and how do these complications differ between individuals with pre-existing conditions and those without?"
seuil = 0.8

# Trouver les questions similaires et leurs réponses
resultats_similaires = trouver_questions_similaires(user_question, seuil)

# Afficher les résultats
print("Question utilisateur :", user_question)
if resultats_similaires:
    for i, (question_similaire, reponse_similaire, score) in enumerate(resultats_similaires, start=1):
        print(f"\nRésultat {i} :")
        print(f"  Question similaire trouvée : {question_similaire}")
        print(f"  Réponse correspondante : {reponse_similaire}")
        print(f"  Score de similarité : {score}")
else:
    print("Aucune question similaire trouvée.")

Question utilisateur : What are the possible long-term health complications caused by the coronavirus, including its effects on the respiratory system, cardiovascular health, and neurological functions, and how do these complications differ between individuals with pre-existing conditions and those without?
Aucune question similaire trouvée.


In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Charger le modèle SBERT pré-entraîné
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encoder les questions du dataset et la question utilisateur en vecteurs sémantiques
corpus_embeddings = model.encode(df["question"].tolist())
user_question = "What causes diabetes?"
user_embedding = model.encode([user_question])

# Calculer la similarité cosinus entre la question utilisateur et le corpus
similarities = cosine_similarity(user_embedding, corpus_embeddings)

# Trouver la question la plus similaire
best_match_idx = similarities.argmax()
best_match_score = similarities[0, best_match_idx]

# Afficher la question et la réponse correspondante
question_similaire = df["question"].iloc[best_match_idx]
reponse_similaire = df["answer"].iloc[best_match_idx]

# Résultats
print(f"Question similaire trouvée : {question_similaire}")
print(f"Réponse correspondante : {reponse_similaire}")
print(f"Score de similarité : {best_match_score}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Question similaire trouvée : What causes Diabetes ?
Réponse correspondante : Type 1 diabetes is an autoimmune disease. In an autoimmune reaction, antibodies, or immune cells, attach to the body's own healthy tissues by mistake, signaling the body to attack them. At present, scientists do not know exactly what causes the body's immune system to attack the insulin-producing cells in the pancreas in people with type 1 diabetes. However, many believe that both genetic factors and environmental factors are involved. Studies now are underway to identify these factors and prevent type 1 diabetes in people at risk. Type 2 diabetesthe most common form of diabetesis caused by a combination of factors, including insulin resistance, a condition in which the bodys muscle, fat, and liver cells do not use insulin effectively. Type 2 diabetes develops when the body can no longer produce enough insulin to compensate for the impaired ability to use insulin.  Get more details about who should be tested f

**Améliorations possibles**
1. **Multilingue :**

- Si le dataset est multilingue ou que l'utilisateur pose des questions dans une autre langue, utilisez un outil de traduction (par exemple, Google Translate) pour traduire les questions utilisateur dans la langue du dataset.
2. **Interface utilisateur :**

- Intégrez ce code dans une interface Streamlit pour une interaction plus fluide.
3. **Enrichissement du dataset :**

- Si certaines questions utilisateur ne trouvent pas de correspondance, ajoutez-les dans le dataset avec une réponse pour améliorer le chatbot.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Charger le modèle SBERT pré-entraîné
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encoder les questions du dataset et la question utilisateur en vecteurs sémantiques
corpus_embeddings = model.encode(df["question"].tolist())
user_question = "Qu'est-ce que la maladie d'Alzheimer ?"
user_embedding = model.encode([user_question])

# Calculer la similarité cosinus entre la question utilisateur et le corpus
similarities = cosine_similarity(user_embedding, corpus_embeddings)

# Trouver la question la plus similaire
best_match_idx = similarities.argmax()
best_match_score = similarities[0, best_match_idx]

# Afficher la question et la réponse correspondante
question_similaire = df["question"].iloc[best_match_idx]
reponse_similaire = df["answer"].iloc[best_match_idx]

# Résultats
print(f"Question similaire trouvée : {question_similaire}")
print(f"Réponse correspondante : {reponse_similaire}")
print(f"Score de similarité : {best_match_score}")

Question similaire trouvée : What causes Alzheimer's Disease ?
Réponse correspondante : There are two types of Alzheimers diseaseearly-onset and late-onset. Early-onset Alzheimers is a rare form of the disease that occurs in people age 30 to 60. It occurs in less than 5 percent of all people with Alzheimers. Almost all people with Alzheimers disease have late-onset Alzheimer's, which usually develops after age 60. Causes Not Fully Understood Scientists do not yet fully understand what causes Alzheimer's disease in most people. In early-onset Alzheimers, a genetic mutation is usually the cause. Late-onset Alzheimers arises from a complex series of brain changes that occur over decades. The causes probably include a mix of genetic, environmental, and lifestyle factors. These factors affect each person differently. Research shows that Alzheimers disease causes changes in the brain years and even decades before the first symptoms appear, so even people who seem free of the disease today ma

In [ ]:
df['question']

,question
0,What is (are) Glaucoma ?
1,What causes Glaucoma ?
2,What are the symptoms of Glaucoma ?
3,What are the treatments for Glaucoma ?
4,What is (are) Glaucoma ?
...,...
16407,What is (are) Diabetic Neuropathies: The Nerve...
16408,How to prevent Diabetic Neuropathies: The Nerv...
16409,How to diagnose Diabetic Neuropathies: The Ner...
16410,What are the treatments for Diabetic Neuropath...
